In [17]:
#Import library
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from google.colab import files


#Load dataset
data = pd.read_csv('/content/Dataset Pengaduan Masyarakat.csv')
data = data[['complaint', 'urgency']]

#Preprocessing: Convert label
y = [ 0 if i=='Not Urgent' else 1 for i in data['urgency'] ]
x = data['complaint']

#Split dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

vocab_size = 500
embedding_dim = 64
max_length = 50
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

#Tokenization
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
wordindex = tokenizer.word_index

train_seq = tokenizer.texts_to_sequences(x_train)
test_seq = tokenizer.texts_to_sequences(x_test)

train_padded = pad_sequences(train_seq, padding=padding_type, truncating=trunc_type, maxlen=max_length)
test_padded = pad_sequences(test_seq, padding=padding_type, truncating=trunc_type, maxlen=max_length)

train_padded = np.array(train_padded)
train_label = np.array(y_train)
test_padded = np.array(test_padded)
test_label = np.array(y_test)

#Sequential model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_padded, train_label, epochs=10, validation_data = (test_padded, test_label))

Epoch 1/10
12/12 [==============================] - 3s 83ms/step - loss: 0.6924 - accuracy: 0.5194 - val_loss: 0.6897 - val_accuracy: 0.5667
Epoch 2/10
12/12 [==============================] - 0s 35ms/step - loss: 0.6885 - accuracy: 0.5750 - val_loss: 0.6853 - val_accuracy: 0.5667
Epoch 3/10
12/12 [==============================] - 0s 36ms/step - loss: 0.6854 - accuracy: 0.5889 - val_loss: 0.6795 - val_accuracy: 0.5889
Epoch 4/10
12/12 [==============================] - 0s 37ms/step - loss: 0.6816 - accuracy: 0.5528 - val_loss: 0.6653 - val_accuracy: 0.6222
Epoch 5/10
12/12 [==============================] - 0s 38ms/step - loss: 0.6496 - accuracy: 0.6250 - val_loss: 0.7992 - val_accuracy: 0.5889
Epoch 6/10
12/12 [==============================] - 0s 36ms/step - loss: 0.6356 - accuracy: 0.6778 - val_loss: 0.6664 - val_accuracy: 0.5889
Epoch 7/10
12/12 [==============================] - 0s 38ms/step - loss: 0.6204 - accuracy: 0.7028 - val_loss: 0.6081 - val_accuracy: 0.6778
Epoch 8/10
12

In [19]:
#Testing with real data
sentence = ["jalanan rusak parah sejak tahun lalu sering terjadi keclekaan, tolong diperbaiki pak"]
sequence = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

predict = model.predict(padded)
predict_value = predict[0][0]

#Urgency Percentage
print(f"Percentage: {predict_value * 100:.1f}%")

#Urgency Value
if predict_value > 0.5:
    print("Urgency = Urgent")
else:
    print("Urgency = Not Urgent")


1/1 [==============================] - 0s 23ms/step
Percentage: 77.5%
Urgency = Urgent


In [20]:
#Save model h5
model.save('new_model77.h5')

#Download file h5
files.download('new_model77.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>